目的
* Webスクレイピングで映画のレビューを収集⇒csvで保存⇒記号を削除

* 前処理されたデータに対しアノテーション作業

* 正解ラベルが付与されたデータセットおよびBERTを用いた感情分析

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Deep learning/sc_nlp/'

Mounted at /content/drive
/content/drive/My Drive/Deep learning/sc_nlp


In [2]:
# Filmarksのスクレイピング
# 対象映画は「Winny」
from bs4 import BeautifulSoup
from urllib import request

# 指定したURLをオープンし、その内容をファイルと同様な手法で取得
url = 'https://filmarks.com/movies/84467/'
response = request.urlopen(url)

# ページのソースコードを取得
soup = BeautifulSoup(response)
response.close()

# ページのソースコードを表示
print(soup)


<!DOCTYPE html>
<html lang="ja"><head><meta charset="utf-8"/><title>Winny - 映画情報・レビュー・評価・あらすじ・動画配信 | Filmarks映画</title><meta content="株式会社つみき" name="author"/><meta content="Winnyの映画情報。6833件のレビュー(口コミ・感想・評価)、内容・ネタバレ、あらすじ、予告編・予告動画、公開映画館情報、公開スケジュール、監督・出演者の関連映画情報、Winnyの動画を配信している動画配信サービスの情報。松本優作監督、東出昌大出演。2002 年、開発者・⾦⼦勇（東出昌⼤）は、簡単にファイルを共有できる⾰新的なソフト「Winny」を開発、試⽤版を「２ちゃんねる」に公開をする。彗星のごとく現れた「Winny」は、本⼈同⼠が直接データのやりとりができるシステムで、瞬く間にシェアを伸ばしていく。しかし、その裏で⼤…" name="description"/><meta content="Winny,映画,感想,動画配信" name="keywords"/><link href="https://filmarks.com/movies/84467" rel="canonical"/><meta content="article" property="og:type"/><meta content="映画『Winny』の感想・レビュー[25040件] | Filmarks" property="og:title"/><meta content="レビュー数：25040件 ／ 平均スコア：★★★★3.8点" property="og:description"/><meta content="https://filmarks.com/movies/84467" property="og:url"/><meta content="https://d2ueuvlup6lbue.cloudfront.net/variants/production/store/pad/240/240/nepi4vzdqy0hs4ktq7ynic4zfnow/_.jpg" property="og:image"/><meta content="h

In [3]:

# レビューゾーンのスクレイピング
# class属性が「p-main-area p-timeline」のdivタグを検索　<div>タグは通常改行を含むブロック要素
p_main_area = soup.find('div', class_='p-main-area p-timeline')
print(p_main_area)



<div class="p-main-area p-timeline"><div class="p-content-detail-highlight"><h2 class="c2-heading-secondary-l">みんなの反応</h2><div class="p-content-detail-highlight__body"><ul><li class="p-content-detail-highlight__item">Winnyの開発問題を巡る刑事訴訟に関する実話を元にした映画</li><li class="p-content-detail-highlight__item">各登場人物たちの信念や正義のために戦う姿が魅力的</li><li class="p-content-detail-highlight__item">日本の凋落を象徴する特異点的事件</li><li class="p-content-detail-highlight__item">裁判の過程にフォーカスした作品で、実力派揃いの演者も魅せる</li><li class="p-content-detail-highlight__item">金子勇氏の天才的な技術者としての生き様が描かれる</li></ul></div><div class="p-content-detail-highlight__robot"><img alt="" class="" height="20" src="https://d2ueuvlup6lbue.cloudfront.net/assets/pc/component/ico_infor_circle_fill-d00713bed51c21196dfa2419e51ca00de9c6885075c9af5d1540c7d575a2b8d8.svg" width="20"/>この情報はユーザーの感想を元に自動生成しています</div></div><div class="p-content-detail-timeline__header"><h3 class="c-heading-5 c-heading-5--with-icon"><img alt="" class="c-heading-5__icon" height="18" src="https://d2ue

In [4]:
# class属性が「p-mark__review」であるdivタグを検索
review = p_main_area.find_all('div', class_='p-mark__review')

for i in range(10):
    print(review[i])



<div class="p-mark__review"><p class="p-mark__review__contents">俳優東出によるWinny事件の真相。<br/><br/>三浦貴大とともに印象的。<br/><br/>当時の事件を覚えていないものの、聞いたことのあったWinny。<br/><br/>背景を少し知ることができた。<br/><br/>技術者に対し、内部通報者に対し、…</p><a href="/movies/84467/reviews/187607409">&gt;&gt;続きを読む</a><ul class="p-mark__tags"><li><a href="/list/tag/FhM_2024" rel="nofollow">#FhM_2024</a></li><li><a href="/list/tag/Winny" rel="nofollow">#Winny</a></li><li><a href="/list/tag/%E6%9D%B1%E5%87%BA%E6%98%8C%E5%A4%A7" rel="nofollow">#東出昌大</a></li><li><a href="/list/tag/%E4%B8%89%E6%B5%A6%E8%B2%B4%E5%A4%A7" rel="nofollow">#三浦貴大</a></li></ul></div>
<div class="p-mark__review"><p class="p-mark__review__contents">敏腕弁護士や警察などが鮮明でキャラ立った登場人物が多く、分かりやすく見やすい映画だった。裁判の論点（金子勇に故意はあったのか）についても、あくまで事実をベースにしつつも観客に受け入れやすい様に工夫さ…</p><a href="/movies/84467/reviews/187583523">&gt;&gt;続きを読む</a></div>
<div class="p-mark__review">東出がよかったし、弁護士の先生とのやりとりも見てて自然でよかった。あと最後の実話の関連が出てくるところは泣いちゃう。</div>
<div class="p-mark__review">良くも悪くも匿名ですぐに情報が回る時代だから不当なことも明るみになりやすくなった。昔は

In [5]:
# class属性が「p-mark__review」であるdivタグを検索
#score = p_main_area.find_all('div', class_='c-rating__score')
score = p_main_area.find_all('div', class_='c2-rating-s__text')
review = p_main_area.find_all('div', class_='p-mark__review')

for i in range(10):
    print(score[i].text)
    print(review[i].text)



3.8
俳優東出によるWinny事件の真相。三浦貴大とともに印象的。当時の事件を覚えていないものの、聞いたことのあったWinny。背景を少し知ることができた。技術者に対し、内部通報者に対し、…>>続きを読む#FhM_2024#Winny#東出昌大#三浦貴大
3.7
敏腕弁護士や警察などが鮮明でキャラ立った登場人物が多く、分かりやすく見やすい映画だった。裁判の論点（金子勇に故意はあったのか）についても、あくまで事実をベースにしつつも観客に受け入れやすい様に工夫さ…>>続きを読む
3.5
東出がよかったし、弁護士の先生とのやりとりも見てて自然でよかった。あと最後の実話の関連が出てくるところは泣いちゃう。
4.2
良くも悪くも匿名ですぐに情報が回る時代だから不当なことも明るみになりやすくなった。昔はどの国も情報が少なく不当が通りやすかったの怖いねえ。
3.6
傍聴席に阿曽山大噴火がおるやーん。小学生時代に他の同級生よりも先にネットに触れてたことでイキっていたことはあれど、世代ではないので最近までWinnyも金子勇さんも知らなかった、無関心。東出の…>>続きを読む
3.8
この時代に生きてたくせに知らなかったから映画にしてくれてありがとうやわ。東出さんええ味出してた〜！途中の警察内部の話は中途半端でもやっとするがまぁ主なとこではないのでよしとして。その時代その時代…>>続きを読む
4.0
インターネット黎明期の実際にあった事件のお話、Winnyは実際に身近にあったし当時ものすごく有名だったことを覚えています。メガネの友だちは鼻を膨らませながらタダで映画や音楽が楽しめると言っていた。…>>続きを読む
-
2024 / 045ドキュメンタリー感ある画。好きでした。でも、なんだか。不完全燃焼な気持ちが否めない。
3.5
良い映画で面白い。5.0＝記憶を消してもう一回見たい 4.5＝絶対観て！お話しよ 4.0＝おすすめ 3.5＝面白い 3.0＝普通
4.0
無知というのは怖い。ただただそう感じた。無知が故に「それ以外」を悪と判断するこの世の中を映画にしたもの。今も尚、この世の中は変わっていない。そんなことを感じた。


In [6]:
import pandas as pd
df_list = []
pages = range(100)

for page in pages:
    url = 'https://filmarks.com/movies/84467?page='+str(page)+''

    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    response.close()

    # レビューゾーンのスクレイピング
    # class属性が「p-main-area p-timeline」のdivタグを検索
    p_main_area = soup.find('div', class_='p-main-area p-timeline')

    # class属性が「p-mark__review」であるdivタグを検索
    #score = p_main_area.find_all('div', class_='c-rating__score')
    score = p_main_area.find_all('div', class_='c2-rating-s__text')
    review = p_main_area.find_all('div', class_='p-mark__review')

    # Determine the shorter list length to avoid IndexError
    min_length = min(len(score), len(review))

    for i in range(min_length): # Iterate over the length of the shorter list
        _df = pd.DataFrame({'score': [score[i].text],
                            'review': [review[i].text]})
        df_list.append(_df)

    print("page%s is over"%page )

# 一つのデータフレームにまとめる
df_review = pd.concat(df_list).reset_index(drop=True)
print(df_review.shape)
df_review.head()

page0 is over
page1 is over
page2 is over
page3 is over
page4 is over
page5 is over
page6 is over
page7 is over
page8 is over
page9 is over
page10 is over
page11 is over
page12 is over
page13 is over
page14 is over
page15 is over
page16 is over
page17 is over
page18 is over
page19 is over
page20 is over
page21 is over
page22 is over
page23 is over
page24 is over
page25 is over
page26 is over
page27 is over
page28 is over
page29 is over
page30 is over
page31 is over
page32 is over
page33 is over
page34 is over
page35 is over
page36 is over
page37 is over
page38 is over
page39 is over
page40 is over
page41 is over
page42 is over
page43 is over
page44 is over
page45 is over
page46 is over
page47 is over
page48 is over
page49 is over
page50 is over
page51 is over
page52 is over
page53 is over
page54 is over
page55 is over
page56 is over
page57 is over
page58 is over
page59 is over
page60 is over
page61 is over
page62 is over
page63 is over
page64 is over
page65 is over
page66 is over
page6

,score,review
0,3.8,俳優東出によるWinny事件の真相。三浦貴大とともに印象的。当時の事件を覚えていないものの、...
1,3.7,敏腕弁護士や警察などが鮮明でキャラ立った登場人物が多く、分かりやすく見やすい映画だった。裁判...
2,3.5,東出がよかったし、弁護士の先生とのやりとりも見てて自然でよかった。あと最後の実話の関連が出て...
3,4.2,良くも悪くも匿名ですぐに情報が回る時代だから不当なことも明るみになりやすくなった。昔はどの国...
4,3.6,傍聴席に阿曽山大噴火がおるやーん。小学生時代に他の同級生よりも先にネットに触れてたことでイキ...


In [7]:
# スコアがない場合は０を入れる
df_review['score'].replace("-", 0 ,inplace=True)

# スコアを数値化
df_review['score'] = df_review['score'].astype(float)


<ipython-input-7-98936d890be4>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_review['score'].replace("-", 0 ,inplace=True)


In [8]:
# スクレイピングしたデータフレームをcsv形式で保存
df_review.to_csv('/content/drive/My Drive/Deep learning/sc_nlp/review.csv', index=False, encoding='utf_8_sig')


In [9]:
import pandas as pd
import re

In [10]:
df = pd.read_csv("/content/drive/My Drive/Deep learning/sc_nlp/review.csv")

In [11]:
df['review'] = df['review'].str.replace('□', '') # 記号を削除
df['review'] = df['review'].str.replace('*', '')
df['review'] = df['review'].str.replace('★', '')
df['review'] = df['review'].str.replace('☆', '')
df['review'] = df['review'].str.replace('😣', '')
df['review'] = df['review'].str.replace('◎', '')
df['review'] = df['review'].str.replace('💻', '')
df['review'] = df['review'].str.replace('🥹', '')
df['review'] = df['review'].str.replace('💢', '')
df['review'] = df['review'].str.replace('😭', '')
df['review'] = df['review'].str.replace('😨', '')
df['review'] = df['review'].str.replace('😶', '')
df['review'] = df['review'].str.replace('✈️', '')
df['review'] = df['review'].str.replace('🥹', '')
df['review'] = df['review'].str.replace('🎞️', '')
df['review'] = df['review'].str.replace('▼', '')
df['review'] = df['review'].str.replace('😱', '')
df['review'] = df['review'].str.replace('✨', '')
df['review'] = df['review'].str.replace('💦', '')
df['review'] = df['review'].str.replace('😊', '')
df['review'] = df['review'].str.replace('👍', '')
df['review'] = df['review'].str.replace('🙏', '')
df['review'] = df['review'].str.replace('🐲', '')
df['review'] = df['review'].str.replace('🐉', '')
df['review'] = df['review'].str.replace('🤩', '')
df['review'] = df['review'].str.replace('⭕️', '')
df['review'] = df['review'].str.replace('🎬', '')
df['review'] = df['review'].str.replace('🫣', '')
df['review'] = df['review'].str.replace('🔪', '')
df['review'] = df['review'].str.replace('😓', '')
df['review'] = df['review'].str.replace('🥲', '')
df['review'] = df['review'].str.replace('⚫︎', '')
df['review'] = df['review'].str.replace('😵‍💫', '')
df['review'] = df['review'].str.replace('🥲', '')
df['review'] = df['review'].str.replace('👏', '')
df['review'] = df['review'].str.replace('⭐️', '')
df['review'] = df['review'].str.replace('◯', '')
df['review'] = df['review'].str.replace('×', '')
df['review'] = df['review'].str.replace('🫶🏻', '')
df['review'] = df['review'].str.replace('☺️', '')

In [12]:
df.to_csv("/content/drive/My Drive/Deep learning/sc_nlp/review_clean.csv", encoding='utf_8_sig')